In [1]:
import os
import yaml
import timeit
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from train_utils import *

In [2]:
project_name = 'Object Detection'
dataset_root = '../Datasets'
output_dir = 'output'
dataset_names = ['airplane']
model_names = ['yolov8n',]
iterations = [1, 1]
epochs = 20
batch_size = 16
image_size = 640
optimizer = 'AdamW'
lr = 1e-2
momentum = 0.9
weight_decay = 1e-4

optim_args = {'optimizer': optimizer, 'lr': lr, 'momentum': momentum, 'weight_decay': weight_decay}

devices = []

if torch.cuda.is_available():
    device = torch.device("cuda:"+str(devices[0]))
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
    
train_split = 0.6
val_split = 0.2
test_split = 0.2

Experiments_Time = datetime.now().strftime("%y%m%d_%H%M%S")
print('Experiment Start Time:',Experiments_Time)
ex_dict = {}
ex_dict['Experiment Time'] = Experiments_Time
ex_dict['Epochs'] = epochs
ex_dict['Batch Size'] = batch_size
ex_dict['Device'] = device
ex_dict['Optimizer'] = optim_args['optimizer']
ex_dict['LR'] = optim_args['lr']
ex_dict['Weight Decay'] = optim_args['weight_decay']
ex_dict['Momentum'] = optim_args['weight_decay']
ex_dict['Image Size'] = image_size
ex_dict['Output Dir'] = output_dir
ex_dict['LRF'] = 1e-4       # Fimal Cosine decay learning rate
ex_dict['Cos LR'] = True    # Apply Cosine Scheduler

# Data Augmentation
ex_dict['hsv_h'] = 0.015
ex_dict['hsv_s'] = 0.7
ex_dict['hsv_v'] = 0.4
ex_dict['degrees'] = 10.0
ex_dict['translate'] = 0.1
ex_dict['scale'] = 0.5
ex_dict['flipud'] = 0.0
ex_dict['fliplr'] = 0.5
ex_dict['mosaic'] = 1.0
ex_dict['mixup'] = 0.5
ex_dict['copy_paste'] = 0.0

# loss weight
loss_config = {
    'yolov5n': {'box': 5.0, 'cls': 0.7, 'dfl': 1.0}, # dfl is not used in YOLOv5 models; ignored during training
    'yolov8n': {'box': 7.5, 'cls': 0.5, 'dfl': 1.5}
}

for iteration in range(iterations[0], iterations[1]+1):
    print(f'(Iter {iteration})')
    seed = iteration
    ex_dict['Iteration'] = iteration
    
    for j, Dataset_Name in enumerate(dataset_names):
        print(f'Dataset: {Dataset_Name} ({j+1}/{len(dataset_names)})'); 
        control_random_seed(seed)
        
        data_yaml_path = f"{dataset_root}/{Dataset_Name}/data_iter_{iteration:02d}.yaml"
        with open(data_yaml_path, 'r') as f:
            data_config = yaml.load(f, Loader=yaml.FullLoader)
            
        ex_dict['Dataset Name'] = Dataset_Name
        ex_dict['Data Config'] = data_yaml_path
        ex_dict['Number of Classes'] = data_config['nc']
        ex_dict['Class Names'] = data_config['names']
        update_dataset_paths(dataset_root, Dataset_Name, iteration)
        
        for k, model_name in enumerate(model_names):
            print(f'{model_name} ({k+1}/{len(model_names)}) (Iter {iteration})', end=' ')
            print(f'Dataset: {Dataset_Name} ({j+1}/{len(dataset_names)})', end=' ')
            control_random_seed(seed)
            
            # Load base model config
            temp_model = YOLO(f'{model_name}.yaml', verbose=False)
            original_model_dict = temp_model.model.yaml

            # Save original yaml
            os.makedirs("models", exist_ok=True)
            original_yaml_path = os.path.join("models", f"{model_name}_original.yaml")
            with open(original_yaml_path, 'w') as f:
                yaml.dump(original_model_dict, f, sort_keys=False)

            # Customize depth/width and modify corresponding scale value
            custom_model_dict = original_model_dict.copy()
            scale_key = model_name.strip()[-1]
            custom_depth = 0.2
            custom_width = 0.25

            # Update scale-specific values
            if 'scales' in custom_model_dict and scale_key in custom_model_dict['scales']:
                custom_model_dict['scales'][scale_key][0] = custom_depth
                custom_model_dict['scales'][scale_key][1] = custom_width

            # Also explicitly add depth_multiple and width_multiple
            custom_model_dict['depth_multiple'] = custom_depth
            custom_model_dict['width_multiple'] = custom_width

            # Save customized yaml
            custom_yaml_path = os.path.join("models", f"{model_name}_custom.yaml")
            with open(custom_yaml_path, 'w') as f:
                yaml.dump(custom_model_dict, f, sort_keys=False)

            # Load modified model
            model = YOLO(custom_yaml_path, verbose=False)
            ex_dict['Model Name'] = model_name
            ex_dict['Model']=model
            model_key = ex_dict['Model Name']
            if model_key in loss_config:
                ex_dict['box'] = loss_config[model_key]['box']
                ex_dict['cls'] = loss_config[model_key]['cls']
                ex_dict['dfl'] = loss_config[model_key]['dfl']
            
            start = timeit.default_timer()
            
            ex_dict = train_model(ex_dict)
            ex_dict = evaluate_model(ex_dict)
            
            ex_dict['Train-Test Time'] = timeit.default_timer() - start
            
            eval_dict = format_measures(ex_dict.get('Test Results'))
            output_csv = f"{ex_dict['Experiment Time']}_{project_name}_Results.csv"
            merge_and_update_df(ex_dict, eval_dict, output_csv, exclude_columns=['Model', 'Train Results', 'Test Results'])


Experiment Start Time: 250416_234105
(Iter 1)
Dataset: airplane (1/1)
yolov8n (1/1) (Iter 1) Dataset: airplane (1/1) engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=../Datasets/airplane/data_iter_01.yaml, epochs=20, time=None, patience=3, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=output/detect, name=250416_234105_yolov8n_airplane_Iter_1, exist_ok=True, pretrained=False, optimizer=AdamW, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_

train: Scanning ../Datasets/airplane/labels... 1051 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1051/1051 [00:00<00:00, 3626.86it/s]


train: New cache created: ../Datasets/airplane/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning ../Datasets/airplane/labels... 351 images, 0 backgrounds, 0 corrupt: 100%|██████████| 351/351 [00:00<00:00, 3658.03it/s]

val: New cache created: ../Datasets/airplane/labels.cache


...

WARNING ⚠️ NMS time limit 3.550s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [01:42<00:00,  9.34s/it]


                   all        351        396     0.0308     0.0278    0.00552    0.00208

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/20      7.15G      2.297      2.837      2.694         57        640:   2%|▏         | 1/66 [00:18<20:16, 18.71s/it]


KeyboardInterrupt: 